In [1]:
from new_model import * 

In [2]:
seed = 10
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
#Data 다운로드 경로지정
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#CIFAR 100 : Data 다운로드 경로지정
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def node_prune(model, pruning_ratio):
    for layer in model.modules():
        if isinstance(layer, M_BasicBlock):
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[0][layer.M_relu1.active != 0]
            cutoff1 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu1.active.data[layer.get_2norm()[0] < cutoff1] *= 0
            
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[1][layer.M_relu2.active != 0]
            cutoff2 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu2.active.data[layer.get_2norm()[1] < cutoff2] *= 0

In [5]:
## scheduler 학습
def fine_tuning(model, mode, EPOCHS):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed10_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [60, 120, 160], gamma = 0.2)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [6]:
###8. test_model
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

## 모델지정

In [11]:
# 모델 지정 및 GPU 지정
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = Wide_ResNet(28, 2, 0.3, 100).to(device)
check_prune = torch.load("../1.Wide_ResNet_model/saved_model/WR_28x2_cifar100.pth")
model.load_state_dict(check_prune['state_dict'])
# test_model(model)

| Wide-Resnet 28x2


<All keys matched successfully>

In [12]:
INITIAL_LR = 0.1*0.2*0.2

MOMENTUM = 0.9

REG = 5e-4

criterion = nn.CrossEntropyLoss()

ratio = 1-0.5**0.19

mode = '28x2_multistepLR_lr0.1*0.2*0.2'

In [9]:
##scheduler 실행 : multstpepLR / 0.1*0.2*0.2
fine_tuning(model, mode = '28x2_CIFAR10_scheduler_multstep_lr0.1*0.2*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 241.32 sec
Validation loss: 0.2539, Validation accuracy: 0.9260
Epoch 20: progress time is 485.25 sec
Validation loss: 0.2605, Validation accuracy: 0.9280
Epoch 30: progress time is 729.55 sec
Validation loss: 0.2277, Validation accuracy: 0.9424
Epoch 40: progress time is 972.6 sec
Validation loss: 0.2362, Validation accuracy: 0.9268
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1213.69 sec
Validation loss: 0.2777, Validation accuracy: 0.9200
Epoch 60: progress time is 1455.61 sec
Validation loss: 0.2564, Validation accuracy: 0.9288
Epoch 70: progress time is 1696.31 sec
Validation loss: 0.2148, Validation accuracy: 0.9404
Epoch 80: progress time is 19

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [24]:
##scheduler 실행 : multstpepLR / 0.1*0.2*0.2
fine_tuning(model, mode = '28x2_CIFAR10_scheduler_multstep_lr0.1*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 239.1 sec
Validation loss: 0.3066, Validation accuracy: 0.9012
Epoch 20: progress time is 480.03 sec
Validation loss: 0.3197, Validation accuracy: 0.9032
Epoch 30: progress time is 720.9 sec
Validation loss: 0.3049, Validation accuracy: 0.9140
Epoch 40: progress time is 959.91 sec
Validation loss: 0.3338, Validation accuracy: 0.8988
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1199.02 sec
Validation loss: 0.2873, Validation accuracy: 0.9072
Epoch 60: progress time is 1437.99 sec
Validation loss: 0.3004, Validation accuracy: 0.9044
Epoch 70: progress time is 1678.23 sec
Validation loss: 0.1964, Validation accuracy: 0.9428
Epoch 80: progress time is 191

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 234.16 sec
Validation loss: 1.1770, Validation accuracy: 0.6940
Epoch 20: progress time is 479.62 sec
Validation loss: 1.1942, Validation accuracy: 0.7016
Epoch 30: progress time is 720.4 sec
Validation loss: 1.2604, Validation accuracy: 0.7004
Epoch 40: progress time is 958.55 sec
Validation loss: 1.2725, Validation accuracy: 0.6868
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1196.95 sec
Validation loss: 1.3298, Validation accuracy: 0.6544
Epoch 60: progress time is 1433.8 sec
Validation loss: 1.2524, Validation accuracy: 0.6764
Epoch 70: progress time is 1672.36 sec
Validation loss: 1.1179, Validation accuracy: 0.7108
Epoch 80: progress time is 1911.

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [9]:
## scheduler : multistepLR , 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 232.15 sec
Validation loss: 1.3737, Validation accuracy: 0.6280
Epoch 20: progress time is 477.82 sec
Validation loss: 1.3429, Validation accuracy: 0.6456
Epoch 30: progress time is 719.38 sec
Validation loss: 1.3015, Validation accuracy: 0.6532
Epoch 40: progress time is 960.24 sec
Validation loss: 1.2716, Validation accuracy: 0.6628
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1201.01 sec
Validation loss: 1.3189, Validation accuracy: 0.6368
Epoch 60: progress time is 1440.86 sec
Validation loss: 1.3199, Validation accuracy: 0.6484
Epoch 70: progress time is 1681.93 sec
Validation loss: 1.0284, Validation accuracy: 0.7136
Epoch 80: progress time is 192

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

## seed 10

In [9]:
##CIFAR10
## scheduler : multistepLR , 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 233.35 sec
Validation loss: 0.0667, Validation accuracy: 0.9780
Epoch 20: progress time is 481.62 sec
Validation loss: 0.0853, Validation accuracy: 0.9728
Epoch 30: progress time is 729.31 sec
Validation loss: 0.0925, Validation accuracy: 0.9740
Epoch 40: progress time is 973.73 sec
Validation loss: 0.1067, Validation accuracy: 0.9680
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1215.65 sec
Validation loss: 0.1687, Validation accuracy: 0.9464
Epoch 60: progress time is 1457.85 sec
Validation loss: 0.1672, Validation accuracy: 0.9496
Epoch 70: progress time is 1695.94 sec
Validation loss: 0.1166, Validation accuracy: 0.9656
Epoch 80: progress time is 193

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [9]:
##CIFAR10
## scheduler : multistepLR , 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 234.6 sec
Validation loss: 0.2503, Validation accuracy: 0.9168
Epoch 20: progress time is 482.27 sec
Validation loss: 0.2909, Validation accuracy: 0.9044
Epoch 30: progress time is 729.42 sec
Validation loss: 0.2995, Validation accuracy: 0.9096
Epoch 40: progress time is 974.46 sec
Validation loss: 0.2824, Validation accuracy: 0.9092
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1218.0 sec
Validation loss: 0.3139, Validation accuracy: 0.8988
Epoch 60: progress time is 1461.85 sec
Validation loss: 0.3329, Validation accuracy: 0.8968
Epoch 70: progress time is 1706.46 sec
Validation loss: 0.1979, Validation accuracy: 0.9396
Epoch 80: progress time is 1952.

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [13]:
##CIFAR100
## scheduler : multistepLR , 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:2'), tensor(896., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:2'), tensor(784., device='cuda:2'))
Epoch 10: progress time is 264.82 sec
Validation loss: 0.5493, Validation accuracy: 0.8432
Epoch 20: progress time is 534.7 sec
Validation loss: 0.5965, Validation accuracy: 0.8252
Epoch 30: progress time is 805.62 sec
Validation loss: 0.7137, Validation accuracy: 0.7912
Epoch 40: progress time is 1073.35 sec
Validation loss: 0.7058, Validation accuracy: 0.8016
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:2'), tensor(684., device='cuda:2'))
Epoch 50: progress time is 1344.69 sec
Validation loss: 1.0524, Validation accuracy: 0.7052
Epoch 60: progress time is 1612.9 sec
Validation loss: 0.9418, Validation accuracy: 0.7372
Epoch 70: progress time is 1882.63 sec
Validation loss: 0.8120, Validation accuracy: 0.7788
Epoch 80: progress time is 2153

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [9]:
##CIFAR100
## scheduler : multistepLR , 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:2'), tensor(896., device='cuda:2'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:2'), tensor(784., device='cuda:2'))
Epoch 10: progress time is 348.17 sec
Validation loss: 1.1196, Validation accuracy: 0.6888
Epoch 20: progress time is 723.41 sec
Validation loss: 1.1208, Validation accuracy: 0.6888
Epoch 30: progress time is 1103.78 sec
Validation loss: 1.3116, Validation accuracy: 0.6520
Epoch 40: progress time is 1482.3 sec
Validation loss: 1.1906, Validation accuracy: 0.6864
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:2'), tensor(684., device='cuda:2'))
Epoch 50: progress time is 1863.29 sec
Validation loss: 1.4103, Validation accuracy: 0.6252
Epoch 60: progress time is 2237.21 sec
Validation loss: 1.3000, Validation accuracy: 0.6560
Epoch 70: progress time is 2642.94 sec
Validation loss: 0.9854, Validation accuracy: 0.7372
Epoch 80: progress time is 30

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

## 결과 확인

In [17]:
## cifar10
check_prune = torch.load("./saved_tuning/28x2_CIFAR10_scheduler_multstep.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

Total examples is 10000, correct examples is 9408; Test accuracy: 0.9408


In [10]:
## cifar10
check_prune = torch.load("./saved_tuning/28x2_CIFAR10_scheduler_multstep_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_CIFAR10_scheduler_multstep finished')

Total examples is 10000, correct examples is 9443; Test accuracy: 0.9443


In [25]:
## cifar10
check_prune = torch.load("./saved_tuning/28x2_CIFAR10_scheduler_multstep_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_CIFAR10_scheduler_multstep finished')

Total examples is 10000, correct examples is 9432; Test accuracy: 0.9432


In [10]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x2_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7164; Test accuracy: 0.7164


In [10]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x2_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7183; Test accuracy: 0.7183


## seed 10

In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR10/28x2_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9352; Test accuracy: 0.9352


In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR10/28x2_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9400; Test accuracy: 0.94


In [14]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x2_multistepLR_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7041; Test accuracy: 0.7041


In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x2_multistepLR_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7233; Test accuracy: 0.7233
